In [1]:
#import
import os

#for operations
import pandas as pd
import numpy as np

#for AWS connection
from boto.s3.connection import S3Connection
from IPython.parallel import Client

In [2]:
#get creds for aws
credentials = pd.read_csv('/home/centos/certificates/credentials.csv')
aws_id = credentials['Access Key Id'][0]
aws_key = credentials['Secret Access Key'][0]

In [3]:
#connect to S3 for data
s3conn = S3Connection( aws_id , aws_key )
bucket = s3conn.get_bucket('avs-kaggle-data')

### Load offers data

In [4]:
offers_keys = bucket.get_all_keys(prefix='offers')
len( offers_keys)

1

In [5]:
offers_keys[0].get_contents_to_filename('offers.csv')

In [6]:
offers_df = pd.read_csv('offers.csv')
print offers_df.shape
offers_df.head(3)

(37, 6)


,offer,category,quantity,company,offervalue,brand
0,1190530,9115,1,108500080,5.00,93904
1,1194044,9909,1,107127979,1.00,6732
2,1197502,3203,1,106414464,0.75,13474


### Load test_hist data

In [7]:
train_hist_keys = bucket.get_all_keys(prefix='trainHistory')
len( train_hist_keys)

1

In [8]:
train_hist_keys[0].get_contents_to_filename('trainHistory.csv')

In [9]:
train_hist_df = pd.read_csv('trainHistory.csv')
print train_hist_df.shape
train_hist_df.head(3)

(160057, 7)


,id,chain,offer,market,repeattrips,repeater,offerdate
0,86246,205,1208251,34,5,t,2013-04-24
1,86252,205,1197502,34,16,t,2013-03-27
2,12682470,18,1197502,11,0,f,2013-03-28


### merge offers and test_hist data

In [10]:
train_hist_offers_df = pd.merge( train_hist_df, offers_df, how='inner')

print train_hist_offers_df.shape
train_hist_offers_df.head(5)

(160057, 12)


,id,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand
0,86246,205,1208251,34,5,t,2013-04-24,2202,1,104460040,2,3718
1,15753725,17,1208251,4,0,f,2013-04-24,2202,1,104460040,2,3718
2,16535563,4,1208251,1,7,t,2013-04-27,2202,1,104460040,2,3718
3,18259179,3,1208251,2,0,f,2013-04-24,2202,1,104460040,2,3718
4,21024070,15,1208251,9,1,t,2013-04-23,2202,1,104460040,2,3718


### import *relevant* transaction data in chunk_rows

In [11]:
trans_keys = bucket.get_all_keys(prefix='transactions.csv.gz')
len( trans_keys)
print trans_keys

[<Key: avs-kaggle-data,transactions.csv.gz>]


In [14]:
trans_keys[0].get_contents_to_filename('transactions.csv.gz')
#this takes a 30-60 seconds when the 3.5gb file is selected

In [ ]:
#create empty df to append to
train_df = pd.DataFrame()

#get headers
col_df = pd.read_csv( 'transactions.csv.gz', 
                     compression='gzip', 
                     #engine='python',
                     nrows=1)
col_headers = list( col_df.columns)

#define variables
chunk_rows = 1000000
rows_hist_train = 349655790 #actual figure, from bash cmd
#rows_hist_train = 5000000 #test

#iterate through transactions
for skip_rows in range( 0, rows_hist_train, chunk_rows):
    
    #test function inputs
    print("importing rows from: %d" %skip_rows) 
    
    #read
    temp_df = pd.read_csv('transactions.csv.gz', 
                          compression= 'gzip', 
                          nrows= chunk_rows,
                          skiprows= skip_rows)
    
    #merge with train_hist_offers to pull ONLY relevant data, then append
    temp_df.columns = col_headers
    temp_df_2 = pd.merge( temp_df, train_hist_offers_df, how='inner')
    train_df = train_df.append( temp_df_2)

#write to csv
train_df.to_csv('train_merged_df.csv') #test


importing rows from: 0
importing rows from: 1000000
importing rows from: 2000000
importing rows from: 3000000
importing rows from: 4000000
importing rows from: 5000000
importing rows from: 6000000
importing rows from: 7000000
importing rows from: 8000000
importing rows from: 9000000
importing rows from: 10000000
importing rows from: 11000000
importing rows from: 12000000
importing rows from: 13000000
importing rows from: 14000000
importing rows from: 15000000
importing rows from: 16000000
importing rows from: 17000000
importing rows from: 18000000
importing rows from: 19000000
importing rows from: 20000000
importing rows from: 21000000
importing rows from: 22000000
importing rows from: 23000000
importing rows from: 24000000
importing rows from: 25000000
importing rows from: 26000000
importing rows from: 27000000

In [ ]:
%%bash
ls
#file is there!

In [ ]:
train_df.info()

In [ ]:
print train_df.shape
train_df.head(5)

In [ ]:
%%bash
aws s3 cp train_merged_df.csv s3://avs-kaggle-data/